This Jupytr notebook 

In [ ]:
# !/bin/sh

# Install required Python packages
python3 -m pip install -r requirements.txt

In [ ]:
import io
import requests
import pandas as pd
import matplotlib.pyplot as plt

Constants for file locations and column mappings used in processing GHCND climate data.

In [ ]:

# Constants to navigate the data
STATE_CODE = "IA" # state code according to ghcnd-states.
STATION_FILE_COLUMNS = ["ID", "LATITUDE", "LONGITUDE", "ELEVATION", "STATE", "NAME", "GSN FLAG", "HCN/CRN FLAG", "WMO ID"] # columns in ghcnd station file (from readme)
STATION_DATA_COLUMN_MAPPING = {
    "ID": [0,10],
    "YEAR": [11,14],
    "MONTH": [15,16],
    "ELEMENT": [17,20],
}

# For downloading data
GHCND_DOWNLOAD_LOCATION = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/all/{}.dly" # location to download data files, format with station ID
GHCND_STATION_FILE_LOCATION = "https://www.ncei.noaa.gov/pub/data/ghcn/daily/ghcnd-stations.txt" # location to download ghcnd station file

# For predownloaded data
GHCND_BASE_LOCATION = "/data/climate_data" # base location of ghcnd data files
STATION_FILE = f"{GHCND_BASE_LOCATION}/ghcnd-stations.csv" # ghcnd station file
OUTPUT_FILE = f"{GHCND_BASE_LOCATION}/output/ghcnd-{STATE_CODE}-data.csv" # output file for state data


Pull data from DLV files

In [ ]:
def load_or_download_file(url, local_path):
    """Load a file from local path or download it if not present."""
    try:
        with open(local_path, 'r') as f:
            data = f.read()
    except FileNotFoundError:
        response = requests.get(url)
        response.raise_for_status()
        data = response.text
        with open(local_path, 'w') as f:
            f.write(data)
    return data

def dlv_to_df(dlv_file):
    """
    Convert a DLV file to a pandas DataFrame.
    1. Load entire DLV file into a DataFrame. With 1 single row per observation.
    2. Split the first column into multiple columns based on fixed width.
    3. Create dataframe with just the columns defined in STATION_DATA_COLUMN_MAPPING
    4. Filter for ELEMENT = TMAX or TMIN by creating two separate dataframes and merging them on MONTH and YEAR.
    5. Calculate TAVG as the average of TMAX and TMIN.
    6. Return the final DataFrame.
    """
    df = pd.read_csv(dlv_file, header=None)
    for col_name, (start, end) in STATION_DATA_COLUMN_MAPPING.items():
        df[col_name] = df[0].str[start:end].str.strip()
    df = df[list(STATION_DATA_COLUMN_MAPPING.keys())]
    tmax_df = df[df["ELEMENT"] == "TMAX"][["YEAR", "MONTH", "ID", "ELEMENT"]].rename(columns={"ELEMENT": "TMAX"})
    tmin_df = df[df["ELEMENT"] == "TMIN"][["YEAR", "MONTH", "ID", "ELEMENT"]].rename(columns={"ELEMENT": "TMIN"})
    df = pd.merge(tmax_df, tmin_df, on=["YEAR", "MONTH", "ID"], how="outer")
    df["TAVG"] = (df["TMAX"].astype(float) + df["TMIN"].astype(float)) / 2
    return df

def get_state_stations(state_code=STATE_CODE):
    """Get list of station IDs for the specified state."""
    station_file = load_or_download_file(
        GHCND_STATION_FILE_LOCATION,
        STATION_FILE
    )
    stations_df = pd.read_csv(station_file)
    stations_df.columns = STATION_FILE_COLUMNS
    state_stations_df = stations_df[stations_df["STATE"] == state_code]
    stations_list = state_stations_df["ID"].tolist()
    return stations_list

def get_state_data(state_code=STATE_CODE):
    """Get climate data for the specified state."""
    stations_list = get_state_stations(state_code)
    all_data_df = pd.DataFrame()
    for station_id in stations_list:
        dlv_file = load_or_download_file(
            GHCND_DOWNLOAD_LOCATION.format(station_id),
            f"{GHCND_BASE_LOCATION}/ghcnd_gsn/{station_id}.dlv"
        )

        try:
            station_data_df = dlv_to_df(dlv_file)
            all_data_df = pd.concat([all_data_df, station_data_df], ignore_index=True)
        except FileNotFoundError:
            print(f"DLV file not found for station {station_id}, skipping.")
    return all_data_df

def get_mean_temperature_per_year(state_code=STATE_CODE):
    """Get mean temperature per year for the specified state."""
    state_data_df = get_state_data(state_code)
    mean_temp_per_year_df = state_data_df.groupby("YEAR")["TAVG"].mean().reset_index()
    mean_temp_per_year_df.rename(columns={"TAVG": "MEAN_TAVG"}, inplace=True)
    return mean_temp_per_year_df


Pull data from crop.csv

In [ ]:
CROP_DATA_BASE_LOCATION = "/data/crop_data" # base location of crop data files
STATE = "IOWA" # state name for crop data

# For downloading data
API_KEY = ""

# For predownloaded data
CROP_DATA_FILE = f"{CROP_DATA_BASE_LOCATION}/crop.csv" # crop data 

In [ ]:
def load_crop_data():
    """Load crop data from CSV file."""
    crop_df = pd.read_csv(CROP_DATA_FILE)
    crop_df = crop_df[
        (crop_df["period"] == "year") &\
        (crop_df["state"] == STATE) &\
        (crop_df["commodity"] == "CORN")
    ]
    crop_df = crop_df[["year", "value"]]
    return crop_df

Merge the data into a single data frame

Draw graphs based on the dataframe to compare averge temperatures to crop yield